In [135]:
from HH_Lukas import HouseholdSpecializationModelClass as HC
from types import SimpleNamespace
import numpy as np  
from scipy import optimize
import ipywidgets as widgets
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import cm

## Question 1

### First, we solve for one set of parameters: 

In [137]:
a = HC()
a.solve_discrete()

a.par.alpha = 0.25
a.par.sigma = 0.5
a_1 = a.solve_discrete()

HF_1 = a_1.HF
HM_1 = a_1.HM

HFHM_1=HF_1/HM_1
print('The ratio of female to male hours worked at home is ', HFHM_1)


The ratio of female to male hours worked at home is  0.3076923076923077


In [132]:
0.5/(0.5-1)

-1.0

### Set parameters to loop through

In [119]:
alphas = np.array([0.25, 0.5, 0.75])
sigmas = np.array([0.5, 1.0, 1.5])
HM = np.zeros(9)
HF = np.zeros(9)


In [122]:


for i, val1 in enumerate(alphas): 
    a.par.alpha = val1
    for j, val2 in enumerate(sigmas,start=1):
        a.par.sigma = val2
        index = i*j
        HM[index] = a.solve_discrete().HM
        HF[index] = a.solve_discrete().HF
        Home_ratios = HF/HM
        
        

/Users/l/Desktop/Programming/Intro_Num-1/HH_Lukas.py:60: RuntimeWarning: invalid value encountered in reciprocal
  power2 = par.sigma/(par.sigma-1)
/var/folders/yf/cbyzbr7d3hl8n98r3rphk8wh0000gn/T/ipykernel_12575/1484860990.py:8: RuntimeWarning: invalid value encountered in true_divide
  Home_ratios = HF/HM


In [123]:
Home_ratios

array([0.30769231, 1.        , 3.25      , 1.        , 3.25      ,
              nan, 3.25      ,        nan,        nan])

In [110]:
for i, val1 in enumerate(alphas): 
    a.par.alpha = val1
    for j, val2 in enumerate(sigmas):
        a.par.sigma = val2
        HM[i*j] = i*j + 1
        HF[i*j] = i*j + 1
        Home_ratios = HF / HM
        

/var/folders/yf/cbyzbr7d3hl8n98r3rphk8wh0000gn/T/ipykernel_12575/449466839.py:7: RuntimeWarning: invalid value encountered in true_divide
  Home_ratios = HF / HM


In [112]:
alphas = np.array([0.25, 0.5, 0.75])
sigmas = np.array([0.5, 1.0, 1.5])
HM = np.zeros(9)
HF = np.zeros(9)

for i, val1 in enumerate(alphas): 
    a.par.alpha = val1
    for j, val2 in enumerate(sigmas):
        a.par.sigma = val2
        index = i * len(sigmas) + j
        HM[index] = i * j + 1
        HF[index] = i * j + 1

Home_ratios = HF / HM

In [113]:
Home_ratios

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [106]:
HF

array([6.5, 4.5, 6.5, 0. , 6.5, 0. , 0. , 0. , 0. ])

In [107]:
HM

array([2. , 4.5, 2. , 0. , 2. , 0. , 0. , 0. , 0. ])

In [108]:
animals = ['cat', 'dog', 'rabbit', 'horse']
for idx, x in enumerate(animals):
    print(idx, x)

0 cat
1 dog
2 rabbit
3 horse
